# OpenCALM SageMaker Finetuning

This is a sample code to finetune and deploy [OpenCALM](https://huggingface.co/spaces/kyo-takano/OpenCALM-7B) with LoRA on SageMaker.

In [ ]:
!pip install -U "sagemaker>=2.143.0"

In [ ]:
import sagemaker, boto3, json
from sagemaker import get_execution_role
from sagemaker.pytorch.model import PyTorchModel
from sagemaker.huggingface import HuggingFace

role = get_execution_role()
region = boto3.Session().region_name
sess = sagemaker.Session()
bucket = sess.default_bucket()

sagemaker.__version__

## Upload Data

Fine Tuning 用の日本語データをフォルダに配置してアップロードする。

ここでは例として [Databricks Dolly 15k](https://github.com/databrickslabs/dolly/tree/master/data) データセットを日本語に翻訳したものを利用します。(License: [Creative Commons Attribution-ShareAlike 3.0 Unported License](https://creativecommons.org/licenses/by-sa/3.0/legalcode))

In [ ]:
!curl -L https://huggingface.co/datasets/kunishou/databricks-dolly-15k-ja/resolve/main/databricks-dolly-15k-ja.json --create-dirs -o ./data/databricks-dolly-15k-ja.json

In [ ]:
!head ./data/databricks-dolly-15k-ja.json

In [ ]:
input_train = sess.upload_data(
    path="./data/databricks-dolly-15k-ja.json",
    key_prefix="Dolly"
)
input_train

また、必要に応じてデータセットを追加してすることも可能です。

必要に応じて以下のコメントを解除して実行してください。その際は、`hyperparameters` の `data_path` をファイル名に合わせてください。

In [ ]:
# !curl -L https://huggingface.co/datasets/kunishou/hh-rlhf-49k-ja/resolve/main/mpt_hhrlhf_49k_ja.json --create-dirs -o ./data/mpt_hhrlhf_49k_ja.json

In [ ]:
# !head ./data/mpt_hhrlhf_49k_ja.json

In [ ]:
# # Merge two json files
# with open('./data/databricks-dolly-15k-ja.json', 'r') as f:
#     dolly_data = json.load(f)
# with open('./data/mpt_hhrlhf_49k_ja.json', 'r') as f:
#     mpt_data = json.load(f)
# new_data = dolly_data + mpt_data
# new_data = [{
#     'instruction': x['instruction'],
#     'input': x['input'],
#     'output': x['output']
# } for x in new_data]

In [ ]:
# with open('./data/dolly-hhrlhf-ja.json', 'w', encoding='utf-8') as f:
#     json.dump(new_data, f, ensure_ascii=False, indent=4)

In [ ]:
# !head ./data/dolly-hhrlhf-ja.json

In [ ]:
# input_train = sess.upload_data(
#     path="./data/dolly-hhrlhf-ja.json",
#     key_prefix="Dolly"
# )
# input_train

## AIO Dataset

In [ ]:
!head -n 2 data/aio_02_train.jsonl

In [ ]:
# Convet .jsonl to .json
import pandas as pd
df = pd.read_json('data/aio_02_train.jsonl', orient='records', lines=True)
df = df.rename(columns={"question": "instruction", "answers": "output"})
df = df[["instruction", "output"]]
# Split to multiple row for each output
# s = df["output"].apply(pd.Series, 1).stack()
# s.index = s.index.droplevel(-1)
# s.name = "output"
# del df["output"]
# df = df.join(s)
# df = df.reset_index(drop=True)
# print(sum(df["output"].apply(lambda x: len(x) > 1)))
df["output"] = df["output"].apply(lambda x: x[0])
df["input"] = ""
# df = df[:2200]
aio = df
print(aio.shape)
df.to_json("data/aio_02_train_formatted.jsonl", orient='records', force_ascii=False, lines=True)

In [ ]:
input_train = sess.upload_data(
    path="./data/aio_02_train_formatted.jsonl",
    key_prefix="OpenCALM"
)
input_train

## Extra Data

In [ ]:
# !head data/aio_02_train_formatted.jsonl

In [ ]:
# !head data/jcommonsense-train-v1.1.json

In [ ]:
# df = pd.read_json('data/jcommonsense-train-v1.1.json', orient='records', lines=True)
# df = df.rename(columns={"question": "instruction"})
# df["output"] = [row["choice" + str(row["label"])] for _, row in df.iterrows()]
# df["input"] = [', '.join([row["choice0"], row["choice1"], row["choice2"], row["choice3"], row["choice4"]]) for _, row in df.iterrows()]
# df = df[["instruction", "output", "input"]]
# jcommonsense = df
# print(jcommonsense.shape)

In [ ]:
# with open("data/jsquad-train-v1.1.json", "r", encoding="utf-8") as f:
#     json_data = json.load(f)
# data = []
# for d in json_data["data"]:
#     title = d["title"]
#     for p in d["paragraphs"]:
#         context = p["context"]
#         for qa in p["qas"]:
#             data.append([
#                 "", # context,
#                 qa["question"],
#                 qa["answers"][0]["text"],
#             ])
# jsquad = pd.DataFrame(data, columns=["input", "instruction", "output"])
# print(jsquad.shape)

In [ ]:
# df = pd.concat([aio, jcommonsense, df_jsquad])
# df.shape
# df.to_json("data/aio_02_train_augmented.jsonl", orient='records', force_ascii=False, lines=True)

In [ ]:
# !head ./data/aio_02_train_augmented.jsonl

In [ ]:
input_train = sess.upload_data(
    path="./data/aio_02_train_augmented.jsonl",
    key_prefix="OpenCALM"
)
input_train

## Fine-tuning

In [ ]:
base_job_name="OpenCALM"
hyperparameters={
    'base_model':'cyberagent/open-calm-7b',
    'load_in_8bit': True,
    # 'load_in_4bit': True,
    'pad_token_id': 1,
    'data_path': '/opt/ml/input/data/train/aio_02_train_formatted.jsonl',
    'num_epochs': 2, # default 3
    'cutoff_len': 256,
    'group_by_length': False,
    'output_dir': '/opt/ml/model',
    # 'resume_from_checkpoint': '/opt/ml/checkpoints',
    'lora_target_modules': '[query_key_value]',
    'lora_r': 16,
    'batch_size': 32,
    'micro_batch_size': 4,
    # 'val_set_size': 200,
    'prompt_template_name': 'alpaca',
}

In [ ]:
huggingface_estimator = HuggingFace(
    base_job_name=base_job_name,
    role=role,
    entry_point='finetune.py',
    source_dir='./scripts/code',
    instance_type='ml.g5.2xlarge',
    instance_count=1,
    volume_size=200,
    transformers_version='4.26',
    pytorch_version='1.13',
    py_version='py39',
    use_spot_instances=True,
    max_wait=86400,
    hyperparameters=hyperparameters,
    metric_definitions=[{'Name': 'eval_loss', 'Regex': "'eval_loss': (\d\.\d+)"},
                        {'Name': 'train_loss', 'Regex': "'loss': (\d\.\d+)"}],
    # checkpoint_s3_uri=f"s3://{bucket}/{base_job_name}/checkpoint/",
)
huggingface_estimator.fit({'train': input_train})

## Download and Extract Model

In [ ]:
import boto3
import sagemaker

def get_latest_training_job_artifact(base_job_name):
    sagemaker_client = boto3.client('sagemaker')
    response = sagemaker_client.list_training_jobs(NameContains=base_job_name, SortBy='CreationTime', SortOrder='Descending')
    training_job_arn = response['TrainingJobSummaries'][0]['TrainingJobArn']
    training_job_description = sagemaker_client.describe_training_job(TrainingJobName=training_job_arn.split('/')[-1])
    return training_job_description['ModelArtifacts']['S3ModelArtifacts']

try:
    model_data = huggingface_estimator.model_data
except:
    # Retrieve artifact url when kernel is restarted
    model_data = get_latest_training_job_artifact('OpenCALM')
    
!aws s3 cp {model_data} opencalm.tar.gz

In [ ]:
!rm -rf scripts/model && mkdir scripts/model
!tar -xvf opencalm.tar.gz -C scripts/model --no-same-owner --wildcards adapter_*
!ls -l scripts/model

## Package and Upload Model

In [ ]:
%cd scripts
!tar -czvf ../package.tar.gz *
%cd -

In [ ]:
model_path = sess.upload_data('package.tar.gz', bucket=bucket, key_prefix=f"OpenCALM")
model_path

## Deploy Model

In [112]:
from sagemaker.async_inference import AsyncInferenceConfig
from sagemaker.serializers import JSONSerializer

endpoint_name = "OpenCALM2"

huggingface_model = PyTorchModel(
    model_data=model_path,
    framework_version="1.13",
    py_version='py39',
    role=role,
    name=endpoint_name,
    env={
        "model_params": json.dumps({
            "base_model": "cyberagent/open-calm-7b",
            "lora_weights": "model", # path relative to model package
            "peft": True,
            "load_8bit": False,
            "prompt_template": "alpaca",
        }),
        "SAGEMAKER_MODEL_SERVER_TIMEOUT": "3600"
    }
)

# deploy model to SageMaker Inference
predictor = huggingface_model.deploy(
    initial_instance_count=1,
    instance_type='ml.g5.2xlarge',
    endpoint_name=endpoint_name,
    serializer=JSONSerializer(),
    # async_inference_config=AsyncInferenceConfig()
)

-!

## Run Inference

In [ ]:
# With SageMaker SDK

from sagemaker.predictor import Predictor
from sagemaker.predictor_async import AsyncPredictor
from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer

predictor_client = Predictor(
    endpoint_name=endpoint_name,
    sagemaker_session=sess,
    serializer=JSONSerializer(),
    deserializer=JSONDeserializer()
)
# predictor_client = AsyncPredictor(
#     predictor=predictor_client,
#     name=endpoint_name
# )
data = {
    "instruction": "ヴァージン・オーストラリアはいつから運航を開始したのですか？",
    "input": "ヴァージン・オーストラリア航空（Virgin Australia Airlines Pty Ltd）の商号で、オーストラリアを拠点とする航空会社です。ヴァージン・ブランドを使用する航空会社の中で、保有機材数では最大の航空会社である。2000年8月31日にヴァージン・ブルーとして、2機の航空機で単一路線で運航を開始した[3]。2001年9月のアンセット・オーストラリアの破綻後、突然オーストラリア国内市場の大手航空会社としての地位を確立した。その後、ブリスベン、メルボルン、シドニーをハブとして、オーストラリア国内の32都市に直接乗り入れるまでに成長した[4]。",
    "max_new_tokens": 256,
    "temperature": 0.3,
    "do_sample": True,
    "pad_token_id": 1,
    "bos_token_id": 0,
    "eos_token_is": 0,
    # "repetition_penalty": 1.05,
    # "top_p": 0.75,
    # "top_k": 40,
    # "no_repeat_ngram_size": 2,
    "stop_ids": [1, 0],
}
response = predictor_client.predict(
    data=data
)
print(response)

In [ ]:
# With Boto3

import boto3
import json

endpoint_name = "OpenCALM"
sagemaker_client = boto3.client('sagemaker-runtime')

data = {
    "instruction": "ヴァージン・オーストラリアはいつから運航を開始したのですか？",
    "input": """ヴァージン・オーストラリア航空（Virgin Australia Airlines Pty Ltd）の商号で、オーストラリアを拠点とする航空会社です。ヴァージン・ブランドを使用する航空会社の中で、保有機材数では最大の航空会社である。2000年8月31日にヴァージン・ブルーとして、2機の航空機で単一路線で運航を開始した[3]。2001年9月のアンセット・オーストラリアの破綻後、突然オーストラリア国内市場の大手航空会社としての地位を確立した。その後、ブリスベン、メルボルン、シドニーをハブとして、オーストラリア国内の32都市に直接乗り入れるまでに成長した[4]。""",
    "max_new_tokens": 128,
    "temperature": 0.7,
    "do_sample": True,
    "pad_token_id": 1,
    "bos_token_id": 0,
    "eos_token_is": 0,
    # "top_p": 0.9,
    # "repetition_penalty": 1.05,
    "stop_ids": [50278, 50279, 50277, 1, 0],
}

response = sagemaker_client.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType='application/json',
    Accept='application/json',
    Body=json.dumps(data)
)

result = json.loads(response['Body'].read())
print(result)

In [ ]:
data = {
    "instruction": "以下の記事によると、昨年生まれた子どもの数は何人でしょうか？",
    "input": """1人の女性が生涯で出産する子どもの数を示す「合計特殊出生率」は去年1年間で「1.26」となり、過去最低となったことが分かりました。去年1年間に生まれた子どもの数も過去最少になっています。
厚労省によりますと、2022年の合計特殊出生率は前の年から0.05ポイント下がって「1.26」でした。
7年連続の減少で、これまでで最も低かった2005年と並び、過去最低となりました。
また、去年1年間に生まれた子どもの数は前の年から4万人余り減って77万747人でした。
子どもが生まれる数は第二次ベビーブームだった1973年以降、減少傾向が続いていて、統計を始めた1899年以来初めて80万人を下回り、過去最少となりました。
一方、去年1年間に死亡した人は156万8961人で、前の年からおよそ12万9000人増え、過去最多となりました。
その結果、死亡した人から生まれた子どもの数を差し引いた人口の減少幅は79万8214人で過去最大となり、人口の減少が加速しています。
厚労省はその要因について、「新型コロナによる“出産控え”や死者数の増加などが影響した可能性がある」と推測しています。また、婚姻の件数は50万4878組で、3年ぶりの増加となりました。""".replace("\n", ""),
    "max_new_tokens": 128,
    "temperature": 0.7,
    "do_sample": True,
    "pad_token_id": 1,
    "bos_token_id": 0,
    "eos_token_is": 0,
    "repetition_penalty": 1.05,
    "stop_ids": [1, 0],
}
for i in range(10):
    response = predictor_client.predict(
        data=data
    )
    print(response)
    print("---")

In [ ]:
def inference(instruction, input):
    data = {
        "instruction": instruction,
        "input": input,
        "max_new_tokens": 64,
        "temperature": 0.1,
        "do_sample": False,
        "num_beams": 5,
        "pad_token_id": 1,
        "bos_token_id": 0,
        "eos_token_is": 0,
        # "repetition_penalty": 1.05,
        "stop_ids": [1, 0],
    }
    response = predictor_client.predict(
        data=data
    )
    return response

import pandas as pd
df = pd.read_json('data/aio_02_dev_v1.0.jsonl', orient='records', lines=True)
correct = 0
answers = []
incorrect = []
for idx, row in df.iterrows():
    result = inference(row['question'], "")
    answers += [result]
    if result in row['answers']:
        correct += 1
    else:
        print(result, row['answers'])
        incorrect += [{ "result": result, "answers": row['answers']}]
print(correct, "/", len(df))

In [ ]:
df_train = pd.read_json('data/aio_02_train.jsonl', orient='records', lines=True)
df_dev = pd.read_json('data/aio_02_dev_v1.0.jsonl', orient='records', lines=True)

In [ ]:
train_questions = set(df_train["original_question"])
dev_questions = set(df_dev["original_question"])
# Check intersection
train_questions.intersection(dev_questions)

train_answers = set(df_train["original_answer"])
dev_answers = set(df_dev["original_answer"])
# Check intersection
print(len(train_answers.intersection(dev_answers)))
common_answer_set = list(train_answers.intersection(dev_answers))
common_answer = list(common_answer_set)
for answer in common_answer[:10]:
    print(answer)
    print(df_train[df_train["original_answer"] == answer]["question"])
    print(df_dev[df_dev["original_answer"] == answer]["question"])
    print("---")

In [ ]:
df = df_train[~df_train["original_answer"].isin(common_answer_set)]
df = df.rename(columns={"question": "instruction", "answers": "output"})
df = df[["instruction", "output"]]
df["output"] = df["output"].apply(lambda x: x[0])
df["input"] = ""
print(len(df))
df.to_json("data/aio_02_train_filtered_formatted.jsonl", orient='records', force_ascii=False, lines=True)

In [ ]:
!head data/aio_02_train_filtered_formatted.jsonl

In [ ]:
input_train = sess.upload_data(
    path="./data/aio_02_train_filtered_formatted.jsonl",
    key_prefix="OpenCALM"
)
input_train

## Benchmark Speed

In [ ]:
%timeit response = predictor_client.predict(data=data)

## Delete Endpoint

In [ ]:
predictor.delete_model()
predictor.delete_endpoint()